In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("yarn").config('spark.executor.instances','100').appName("chiang_ct").getOrCreate()

## Merge

In [3]:
df_cross_all = spark.read.csv("/user/spark/share/output/table2/cross/tb2_cross_all",header="true")

In [9]:
df_cross_all.columns

['user_cate',
 'user_cate_pv1',
 'pv_1',
 'user_cate_cart1',
 'cart_1',
 'user_cate_fav1',
 'fav_1',
 'user_cate_buy1',
 'buy_1',
 'user_cate_pv2',
 'pv_2',
 'user_cate_cart2',
 'cart_2',
 'user_cate_fav2',
 'fav_2',
 'user_cate_buy2',
 'buy_2',
 'user_cate_pv3',
 'pv_3',
 'user_cate_cart3',
 'cart_3',
 'user_cate_fav3',
 'fav_3',
 'user_cate_buy3',
 'buy_3']

In [4]:
df_cross_all = df_cross_all.fillna("0")

In [5]:
df_cross_all.createOrReplaceTempView('df_cross_all_table')

In [7]:
#choose only those columns we need
df_cross_all_table = spark.sql("""select user_cate as user_cate_c, pv_1, cart_1, fav_1, buy_1, \
                            pv_2, cart_2, fav_2, buy_2, pv_3, cart_3, fav_3, buy_3  from df_cross_all_table""")

In [10]:
df_cross_all_table.show()

+-------------+----+------+-----+-----+----+------+-----+-----+----+------+-----+-----+
|  user_cate_c|pv_1|cart_1|fav_1|buy_1|pv_2|cart_2|fav_2|buy_2|pv_3|cart_3|fav_3|buy_3|
+-------------+----+------+-----+-----+----+------+-----+-----+----+------+-----+-----+
| 1000007_4384|   0|     0|    0|    0|   2|     0|    0|    0|   0|     0|    0|    0|
|  1000009_589|   0|     0|    0|    0|   1|     0|    0|    0|   0|     0|    0|    0|
|1000011_12698|   0|     0|    0|    0|   0|     0|    0|    0|   3|     1|    0|    0|
|1000012_10725|   0|     0|    0|    0|   2|     0|    0|    0|   0|     0|    0|    0|
| 1000014_7146|   0|     0|    0|    0|   1|     0|    0|    0|   1|     0|    0|    0|
|   1000019_10|   0|     0|    0|    0|   0|     0|    0|    0|   8|     1|    0|    0|
| 1000026_9101|   0|     0|    0|    0|   8|     0|    0|    0|   0|     0|    0|    0|
| 1000027_5954|   0|     0|    0|    0|   0|     0|    0|    0|   1|     0|    0|    0|
| 1000029_5944|   0|     0|    0

In [12]:
from pyspark.sql.functions import isnull
for col in df_cross_all_table:
    print(df_cross_all_table.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user_cate_c'>
0 -----> Column<b'pv_1'>
0 -----> Column<b'cart_1'>
0 -----> Column<b'fav_1'>
0 -----> Column<b'buy_1'>
0 -----> Column<b'pv_2'>
0 -----> Column<b'cart_2'>
0 -----> Column<b'fav_2'>
0 -----> Column<b'buy_2'>
0 -----> Column<b'pv_3'>
0 -----> Column<b'cart_3'>
0 -----> Column<b'fav_3'>
0 -----> Column<b'buy_3'>


### join cross with merge

In [13]:
df_merge = spark.read.csv("/user/spark/share/output/merge_nc",header=True)

In [15]:
table2 = df_merge.join(df_cross_all_table,(df_merge.user_cate == df_cross_all_table.user_cate_c) ,'left_outer')

In [17]:
table2.columns

['user',
 'time_stamp',
 'adgroup_id',
 'pid',
 'clk',
 'cate_id',
 'campaign_id',
 'customer',
 'brand',
 'price',
 'cms_segid',
 'cms_group_id',
 'final_gender_code',
 'age_level',
 'pvalue_level',
 'shopping_level',
 'occupation',
 'new_user_class_level',
 'user_cate',
 'user_cate_c',
 'pv_1',
 'cart_1',
 'fav_1',
 'buy_1',
 'pv_2',
 'cart_2',
 'fav_2',
 'buy_2',
 'pv_3',
 'cart_3',
 'fav_3',
 'buy_3']

In [18]:
table2_drop = table2.drop('user_cate_c')

In [19]:
for col in table2_drop:
    print(table2_drop.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user'>
0 -----> Column<b'time_stamp'>
0 -----> Column<b'adgroup_id'>
0 -----> Column<b'pid'>
0 -----> Column<b'clk'>
0 -----> Column<b'cate_id'>
0 -----> Column<b'campaign_id'>
0 -----> Column<b'customer'>
0 -----> Column<b'brand'>
0 -----> Column<b'price'>
1528526 -----> Column<b'cms_segid'>
1528526 -----> Column<b'cms_group_id'>
1528526 -----> Column<b'final_gender_code'>
1528526 -----> Column<b'age_level'>
14552798 -----> Column<b'pvalue_level'>
1528526 -----> Column<b'shopping_level'>
1528526 -----> Column<b'occupation'>
8224829 -----> Column<b'new_user_class_level'>
0 -----> Column<b'user_cate'>
1141355 -----> Column<b'pv_1'>
1141355 -----> Column<b'cart_1'>
1141355 -----> Column<b'fav_1'>
1141355 -----> Column<b'buy_1'>
1141355 -----> Column<b'pv_2'>
1141355 -----> Column<b'cart_2'>
1141355 -----> Column<b'fav_2'>
1141355 -----> Column<b'buy_2'>
1141355 -----> Column<b'pv_3'>
1141355 -----> Column<b'cart_3'>
1141355 -----> Column<b'fav_3'>
1141355 -----> Column<

In [21]:
table2_drop.count()

26557961

In [23]:
table2_drop.write.csv("hdfs:///user/spark/share/output/table2/lovefengyi/table_2",header="true")

## fill na

In [37]:
cross = ['pv_1', 'cart_1', 'fav_1', 'buy_1', 'pv_2', 'cart_2', 'fav_2', 'buy_2', 'pv_3', 'cart_3', 'fav_3', 'buy_3']
table2_f = table2_drop
for col in cross:
    table2_f = table2_f.fillna({col:0})

In [38]:
print(table2_f.filter(isnull('cart_1')).count())

0


In [39]:
table2_f = table2_f.fillna('999999999')

In [45]:
table2_f.filter(table2_f.cart_1==999999999).count()

0

In [46]:
table2_f.write.csv("hdfs:///user/spark/share/output/table2/lovefengyi/table_2_f",header="true")

In [47]:
table2_f.count()

26557961

In [48]:
for col in table2_f:
    print(table2_f.filter(isnull(col)).count(),"----->",col)

0 -----> Column<b'user'>
0 -----> Column<b'time_stamp'>
0 -----> Column<b'adgroup_id'>
0 -----> Column<b'pid'>
0 -----> Column<b'clk'>
0 -----> Column<b'cate_id'>
0 -----> Column<b'campaign_id'>
0 -----> Column<b'customer'>
0 -----> Column<b'brand'>
0 -----> Column<b'price'>
0 -----> Column<b'cms_segid'>
0 -----> Column<b'cms_group_id'>
0 -----> Column<b'final_gender_code'>
0 -----> Column<b'age_level'>
0 -----> Column<b'pvalue_level'>
0 -----> Column<b'shopping_level'>
0 -----> Column<b'occupation'>
0 -----> Column<b'new_user_class_level'>
0 -----> Column<b'user_cate'>
0 -----> Column<b'pv_1'>
0 -----> Column<b'cart_1'>
0 -----> Column<b'fav_1'>
0 -----> Column<b'buy_1'>
0 -----> Column<b'pv_2'>
0 -----> Column<b'cart_2'>
0 -----> Column<b'fav_2'>
0 -----> Column<b'buy_2'>
0 -----> Column<b'pv_3'>
0 -----> Column<b'cart_3'>
0 -----> Column<b'fav_3'>
0 -----> Column<b'buy_3'>
